# Финальный проект - Космические братья

## Введение

**Описание проекта**

Мы занимаемся аналитикой для проекта Космические братья. Это новая игра, в которой пользователи могут строить разные здания и сражаться с другими пользователями.

Отдел маркетинга попросил нас разобраться в источниках, которые мы используем для привлечения пользователей. В долгосрочном периоде мы хотим оптимизировать расходы на рекламу, поэтому нам необходимо определить какие источники работают хорошо, а какие источники - плохо.

Пока что в игре еще не реализована монетизация, но мы планируем зарабатывать на пользователях, показывая рекламу во время стороительства зданий. Для определения эффективных каналов мы можем использовать промежуточные метрики, например: время в игре, количество построенных зданий, факт завершения первого уровня, и пр.

[Презентация проекта](https://drive.google.com/file/d/1rc0AeGIkDvpBb1nVBw3oKH2XrT1LVdro/view?usp=sharing)

**Описание данных**

Датасет `game_actions.csv` - в нем представлены данные по игре пользователей на первом уровне. Завершение первого уровня требует от игрока выполнения одного из двух условий: 1) победа над первым врагом (PvP) 2) реализация проекта - разработка орбитальной сборки спутников (PvE).

В датасете содержатся данные первых пользователей приложения — когорты пользователей, которые начали пользоваться приложением в период с 4 по 10 мая включительно. Можем предположить, что день, когда пользователь открыл приложение - это день, когда он увидел рекламу.

1. `event_datetime` — время события
2. `event` — одно из трёх событий
3. `building_type` — один из трёх типов здания
4. `user_id` — идентификатор пользователя
5. `project_type` — тип реализованного проекта

Помимо основного датасета есть два датасета с информацией о рекламных активностях. Они также помогут в решении задачи.

Датасет `ad_costs.csv` - в нем представлены затраты на привлечения пользователей по дням и источникам. Чтобы привлечь пользователей 4 мая, реклама была оплачена 3 мая и т.д.

1. `day` - день, в который был совершен клик по объявлению
2. `source` - источник трафика
3. `cost` - стоимость кликов

Датасет `user_source.csv` содержит колонки:

1. `user_id` - идентификатор пользователя
2. `source` - источников, с которого пришёл пользователь, установивший приложение

**План работы**

1. Загрузить и подготовить данные
   1. Почистить колонки, задать типы данных
   2. Обработать пропуски и дубликаты
   3. Найти и исправить ошибки (если есть) 
2. Провести исследовательский анализ и рассчитать
   1. Количество (и доли) пользователей за весь период, тех кто прошел игру, время в игре
   2. Количество (и доли на пользователя) событий за весь период, реализованных проектов, построенных зданий
   3. Количество (и доли) пользователей по источникам трафика, стоимость привлечения
3. Проверить гипотезы
   1. Время завершения уровня различается в зависимости от способа прохождения
   2. Количество построек для Yandex и Facebook отличается
4. Сделать выводы и рекомендации


In [2]:
import pandas as pd
import numpy as np
import plotly.express as px
from scipy import stats as st
from IPython.display import display

# Save raw datasets in case we need them
try:
    raw_actions = pd.read_csv('game_actions.csv')
    raw_costs = pd.read_csv('ad_costs.csv')
    raw_sources = pd.read_csv('user_source.csv')
except:
    raw_actions = pd.read_csv('/datasets/game_actions.csv')
    raw_costs = pd.read_csv('/datasets/ad_costs.csv')
    raw_sources = pd.read_csv('/datasets/user_source.csv')

# Constants, which we will need later
FIG_WIDTH = 8
FIG_HEIGHT = 5


## Предобработка данных

Для начала проверим какие данные нам достались. Нас интересуют:

1. Названия колонок.
2. Неподходящие под тип данных значения в колонках.
3. Артефакты в данных и повторяющиеся строки.
4. Пропущенные значения.
5. Дополнительные колонки и типы данных.

Пойдем по порядку.

In [3]:
# Check columns
for dataset in [raw_actions, raw_costs, raw_sources]:
    print('-' * 50)
    dataset.info()

--------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 135640 entries, 0 to 135639
Data columns (total 5 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   event_datetime  135640 non-null  object
 1   event           135640 non-null  object
 2   building_type   127957 non-null  object
 3   user_id         135640 non-null  object
 4   project_type    1866 non-null    object
dtypes: object(5)
memory usage: 5.2+ MB
--------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28 entries, 0 to 27
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   source  28 non-null     object 
 1   day     28 non-null     object 
 2   cost    28 non-null     float64
dtypes: float64(1), object(2)
memory usage: 800.0+ bytes
--------------------------------------------------
<class 'pandas.core.frame.DataFrame'>


Название колонок уже заданы нормально. У нас есть много пропущенных занчений в датасете `raw_actions`, но это может быть связано с тем, как были собраны данные. Мы на это посмотрим в следующих секциях. Наконец, надо будет проверить типы данных.

In [4]:
for column in ['event', 'building_type', 'project_type']:
    print(
        '-' * 50, '\n', 
        raw_actions[column].value_counts(),
        sep=''
    )

--------------------------------------------------
building            127957
finished_stage_1      5817
project               1866
Name: event, dtype: int64
--------------------------------------------------
spaceport          59325
assembly_shop      54494
research_center    14138
Name: building_type, dtype: int64
--------------------------------------------------
satellite_orbital_assembly    1866
Name: project_type, dtype: int64


In [5]:
print(
    '-' * 50, '\n',
    raw_costs.source.value_counts(), '\n',
    '-' * 50, '\n',
    raw_sources.source.value_counts(),
    sep='',
)

--------------------------------------------------
facebook_ads               7
instagram_new_adverts      7
yandex_direct              7
youtube_channel_reklama    7
Name: source, dtype: int64
--------------------------------------------------
yandex_direct              4817
instagram_new_adverts      3347
facebook_ads               2726
youtube_channel_reklama    2686
Name: source, dtype: int64


У нас нету странных значений в записях: каналы в затратах совпадают с каналами в источниках, а события в `row_actions` адекватны.

Следующим шагом, проверим дубликаты.

In [6]:
for dataset in [raw_actions, raw_sources]:
    display(dataset[dataset.duplicated()])


,event_datetime,event,building_type,user_id,project_type
74891,2020-05-10 18:41:56,building,research_center,c9af55d2-b0ae-4bb4-b3d5-f32aa9ac03af,NaN


,user_id,source


У нас есть одинокая строка с полным дубликатом в `raw_actions`. Убьем ее: явно один и тот же пользватель не мог совершить одно и то же действия в одно и то же время.

Посмотрим на пропущенные значения. У нас две колонки с пропущенными значениями - `building_type` и `project_type`.

In [7]:
(raw_actions
 .fillna('NA')
 .pivot_table(index=['event', 'building_type','project_type'], values='user_id', aggfunc='count')
 .reset_index()
 .rename(columns={0:'count'})
)

,event,building_type,project_type,user_id
0,building,assembly_shop,NA,54494
1,building,research_center,NA,14138
2,building,spaceport,NA,59325
3,finished_stage_1,NA,NA,5817
4,project,NA,satellite_orbital_assembly,1866


Похоже, что все пропущенные значения на своих местах: `project_type` содержит NAs, когда игрок строил здания или закончил уровень. Действительно, в этих событиях не было проекта, а значит и колонка должна быть пустой.

`buidling_type` содержит пропуски, когда игрок завершил уровень или завершил проект. Здесь тоже все логично: завершение уровня не является постройкой, а проект, судя по названию и событиям, тоже не является зданием.

Причешем наши датасеты.

In [8]:
# No changes to the raw datasets
df_sources = raw_sources.copy()

# Changes in the raw datasets
df_costs = raw_costs.copy().astype({'day': 'datetime64[D]'})

df_actions = (
    raw_actions.copy()
    .drop_duplicates()
    .assign(
        event_day=lambda df: df.event_datetime,
        event=lambda df: df.event.replace({
            'building': 'building_completed',
            'finished_stage_1': 'stage_1_completed',
            'project': 'project_completed',
        }),
        building_type=lambda df: df.building_type.fillna('no_building'),
        project_type=lambda df: df.project_type.fillna('not_a_project')
    )
    .astype({'event_datetime': 'datetime64[s]', 'event_day': 'datetime64[D]'})
)

## Исследовательский анализ данных

Посмотрим, какие основные зависимости в данных мы можем найти: сколько у нас пользователей, откуда они приходят, как себя ведут в игре и прочее.

### Анализ количества пользователей

In [9]:
print('Total number of unique users: ', df_actions.user_id.nunique())

df_temp = (
    pd.pivot_table(
        data=df_actions,
        index='event_day',
        values='user_id',
        aggfunc='nunique'
    )
    .reset_index()
    .set_axis(['day_event', 'user_id_count'], axis=1)
    .rename_axis('index', axis=1)
)

fig = px.bar(
    df_temp,
    x='day_event',
    y='user_id_count',
    title='Total number of unique users per day',
    labels=dict(user_id_count='Unique users, #', day_event='Date'),
    width=FIG_WIDTH * 100,
    height=FIG_HEIGHT * 100,
    template='plotly_white',
)
fig.update_layout(legend=dict(orientation='h', yanchor='top', y=1.1, x=0))
fig.show()

Total number of unique users:  13576


Всего в нашей выборке 13.5К уникальных пользователей. После 10 мая количество игроков в день падает с 9К до 6К. Вероятно, это связано с тем, что мы рассматриваем игроков, привлеченных рекламой с 3 по 9 мая.

In [10]:
df_temp = (
    pd.pivot_table(
        data=df_actions[df_actions.event != 'building_completed'],
        index='event_day',
        values='user_id',
        aggfunc='nunique',
    )
    .reset_index()
    .set_axis(['day_event', 'user_id_count'], axis=1)
    .rename_axis('index', axis=1)
)

fig = px.bar(
    df_temp,
    x='day_event',
    y='user_id_count',
    title='Total number of unique users who completed the stage per day',
    labels=dict(user_id_count='Unique users, #', day_event='Date'),
    width=FIG_WIDTH * 100,
    height=FIG_HEIGHT * 100,
    template='plotly_white',
)
fig.update_layout(
    legend=dict(orientation='h', yanchor='top', y=1.1, x=0), hovermode='x unified'
)
fig.show()


Ожидаемо, у нас есть игроки, которые закончили игру очень рано, а есть игроки, которые заканчиывают позже.

Посмотрим, сколько всего игроков прошло первый уровень.

In [11]:
df_temp = (
    pd.pivot_table(
        data=df_actions[df_actions.event != 'building_completed'],
        columns='event',
        values='user_id',
        aggfunc='nunique',
    )
    .assign(won_by_pvp=lambda df: df.stage_1_completed - df.project_completed)
    .set_axis(['won_by_completing_project', 'All', 'won_by_pvp'], axis=1)
    .T.reset_index()
    .rename_axis('index', axis=1)
    .set_axis(['win_type', 'user_id_count'], axis=1)
    .sort_values('user_id_count')
    .assign(
        event_per = lambda df: round(100 * df.user_id_count / df.user_id_count.iloc[-1], 1),
        sort_key = lambda df: df.win_type == 'All',
    )
    .sort_values(['sort_key', 'event_per'], ascending=[True, False])
    .drop('sort_key', axis=1)
)

fig = px.bar(
    df_temp[df_temp.win_type != 'All'],
    x='user_id_count',
    y='win_type',
    text_auto=True,
    title='Total number of unique users who completed the stage by win type',
    labels=dict(user_id_count='Unique users, #', win_type='Win type'),
    width=FIG_WIDTH * 100,
    height=FIG_HEIGHT * 100,
    template='plotly_white',
)
fig.update_xaxes(showticklabels=False, showgrid=False)
fig.update_layout(
    legend=dict(orientation='h', yanchor='top', y=1.1, x=0),
    yaxis={'categoryorder': 'total ascending'},
)
fig.show()

fig = px.pie(
    df_temp[df_temp.win_type != 'All'],
    values='user_id_count',
    names='win_type',
    title='Percentage of unique users who completed the stage by win type',
    width=FIG_WIDTH * 100,
    height=FIG_HEIGHT * 100,
    template='plotly_white',
)
fig.show()

display(df_temp)


,win_type,user_id_count,event_per
2,won_by_pvp,3951,67.9
0,won_by_completing_project,1866,32.1
1,All,5817,100.0


Только треть игроков прошла уровень через заверешение проекта. Большинство игроков безжалостно уничтожили своих соперников.

Посмотрим на время, которое пользователи провели в игре.

In [12]:
df_temp = (
    pd.pivot_table(
        data=df_actions,
        index='user_id',
        values='event_datetime',
        aggfunc=['min', 'max'],
    )
    .reset_index()
    .droplevel(level=1, axis=1)
    .rename_axis('index', axis=1)
    .set_axis(['user_id', 'start_datetime', 'end_datetime'], axis=1)
    .assign(
        hours_active = lambda df: (df.end_datetime - df.start_datetime).dt.total_seconds() / 60 / 60
    )
)

fig = px.violin(
    df_temp,
    x='hours_active',
    # spanmode='hard',
    title='Distribution of # of hours active',
    labels=dict(hours_active='Hours active'),
    width=FIG_WIDTH * 100,
    height=FIG_HEIGHT * 100,
    template='plotly_white',
)
fig.show()

display(round(df_temp.describe().T, 1))


,count,mean,std,min,25%,50%,75%,max
hours_active,13576.0,256.2,104.4,0.0,184.6,252.5,319.5,742.0


В среднем, игроки находились на первом уровне около 250 часов. Дальше мы посмотрим, как это число меняется в зависимости от канала привлечения.

### Анализ количества событий

Следующим шагом посмотрим, как пользователи вели себя в игре.

In [13]:
print(
    'Total number of events: ',
    df_actions.event.count(),
    '\n',
    'Average number of events per unique user: ',
    round(df_actions.event.count() / df_actions.user_id.nunique(), 2),
    sep='',
)

df_temp = (
    pd.pivot_table(
        data=df_actions, index=['event_day', 'event'], values='user_id', aggfunc='count'
    )
    .reset_index()
    .set_axis(['day_event', 'event', 'event_count'], axis=1)
    .rename_axis('index', axis=1)
    .assign(
        event_per = lambda df: round(100 * df.event_count / df.event_count.iloc[-1], 1),
        sort_key = lambda df: df.event == 'All',
    )
    .sort_values(['sort_key', 'event_per'], ascending=[True, False])
    .drop('sort_key', axis=1)
)

fig = px.bar(
    df_temp,
    x='day_event',
    y='event_count',
    color='event',
    title='Total number of events by type per day',
    labels=dict(event_count='Events, #', day_event='Date'),
    width=FIG_WIDTH * 100,
    height=FIG_HEIGHT * 100,
    template='plotly_white',
)
fig.update_layout(
    legend=dict(orientation='h', yanchor='top', y=1.1, x=0), hovermode='x unified'
)
fig.show()


Total number of events: 135639
Average number of events per unique user: 9.99


В нашей игре уже достаточно приличное количество пользователей, которые, видимо, довольно много кликают - в среднем на пользователя приходится по 10 событий. Видимо, многие из них строят несколько космопортов или сборочных цехов за раз.

Другое интересное наблюдение - количество событий в день. У нас в начале периода явно есть много активности - игроки строят здания. C 10 на 11 мая у нас начинают появляться участники, которые завершили уровень.

In [14]:
df_temp = (
    pd.pivot_table(
        data=df_actions[df_actions.building_type != 'no_building'],
        index='building_type',
        values='user_id',
        aggfunc='count',
        margins=True,
    )
    .reset_index()
    .set_axis(['event', 'event_count'], axis=1)
    .rename_axis('index', axis=1)
    .assign(
        event_per = lambda df: round(100 * df.event_count / df.event_count.iloc[-1], 1),
        sort_key = lambda df: df.event == 'All',
    )
    .sort_values(['sort_key', 'event_per'], ascending=[True, False])
    .drop('sort_key', axis=1)
)

fig = px.bar(
    df_temp[df_temp.event != 'All'],
    x='event_count',
    y='event',
    text_auto=True,
    title='Total number of events by buidling type',
    labels=dict(event_count='Events, #', event='Building type'),
    width=FIG_WIDTH * 100,
    height=FIG_HEIGHT * 100,
    template='plotly_white',
)
fig.update_xaxes(showticklabels=False, showgrid=False)
fig.update_layout(yaxis={'categoryorder': 'total ascending'})
fig.show()

display(df_temp)


index,event,event_count,event_per
2,spaceport,59325,46.4
0,assembly_shop,54494,42.6
1,research_center,14137,11.0
3,All,127956,100.0


Похоже, игроки гораздо чаще строят `spaceport` и `assembly_shop`, чем `research_center`. Это может быть связно с механикой игры (например, только один исследовательский центр разрешен за игру), так и со стратегиями, которые выбирают игроки (зерг раш!).

Посмотрим, какое количество событий происходило на пользователя по дням.

In [15]:
df_temp = (
    pd.pivot_table(
        data=df_actions,
        index='event_day',
        values='user_id',
        aggfunc=['count', 'nunique'],
    )
    .reset_index()
    .set_axis(['day_event', 'event_count', 'user_id_count'], axis=1)
    .rename_axis('index', axis=1)
    .assign(
        event_avg_per_user = lambda df: round(df.event_count / df.user_id_count, 1),
    )
)

fig = px.bar(
    df_temp,
    x='day_event',
    y='event_avg_per_user',
    # color='event',
    title='Average number of events per user per day',
    labels=dict(event_avg_per_user='Average events per user, #', day_event='Date'),
    width=FIG_WIDTH * 100,
    height=FIG_HEIGHT * 100,
    template='plotly_white',
)
fig.show()


Среднее количество событий на пользователя в день относительно стабильно. Ну и хорошо.

### Анализ источников трафика

Посмотрим, откуда пришли наши пользователи.

In [16]:
df_temp = (
    pd.pivot_table(
        data=df_sources,
        index='source',
        values='user_id',
        aggfunc='nunique',
        margins=True,
    )
    .reset_index()
    .set_axis(['source', 'user_id_count'], axis=1)
    .rename_axis('index', axis=1)
    .assign(
        user_per = lambda df: round(100 * df.user_id_count / df.user_id_count.iloc[-1], 1),
        sort_key = lambda df: df.source == 'All',
    )
    .sort_values(['sort_key', 'user_per'], ascending=[True, False])
    .drop('sort_key', axis=1)
)

fig = px.bar(
    df_temp[df_temp.source != 'All'],
    x='user_id_count',
    y='source',
    text_auto=True,
    title='Total number of unique users by traffic source',
    labels=dict(user_id_count='Number of unique users, #', source='Traffic source'),
    width=FIG_WIDTH * 100,
    height=FIG_HEIGHT * 100,
    template='plotly_white',
)
fig.update_xaxes(showticklabels=False, showgrid=False)
fig.update_layout(yaxis={'categoryorder': 'total ascending'})
fig.show()

fig = px.pie(
    df_temp[df_temp.source != 'All'],
    values='user_id_count',
    names='source',
    title='Percentage of unique users by traffic source',
    width=FIG_WIDTH * 100,
    height=FIG_HEIGHT * 100,
    template='plotly_white',
)
fig.show()

display(df_temp)


index,source,user_id_count,user_per
2,yandex_direct,4817,35.5
1,instagram_new_adverts,3347,24.7
0,facebook_ads,2726,20.1
3,youtube_channel_reklama,2686,19.8
4,All,13576,100.0


Чуть больше чем треть наших игроков пришли за счет `yandex_direct`. Остальные игроки равномерно распределены между другими каналами.

Посмотрим, как мы тратили деньги на пользователей.

In [17]:
df_temp = (
    pd.pivot_table(
        data=df_costs,
        index='day',
        columns='source',
        values='cost',
        aggfunc='sum',
        margins=True,
    )
    .reset_index()
    .rename_axis('index', axis=1)
    .set_axis(['day', 'cost_fb', 'cost_ig', 'cost_ya', 'cost_yt', 'cost_total'], axis=1)
    # .apply(
    #     lambda df: df.iloc[:, 1:].div(df.cost_total, axis=0)
    # )
)

df_temp.iloc[:, 1:] = round(
    100 * df_temp.iloc[:, 1:].div(df_temp.cost_total, axis=0), 2
)

fig = px.bar(
    round(df_costs, 0),
    x='day',
    y='cost',
    color='source',
    text_auto=True,
    title='Ad cost per source per day',
    labels=dict(cost='Cost, $', day='Date'),
    width=FIG_WIDTH * 100,
    height=FIG_HEIGHT * 100,
    template='plotly_white',
)
fig.update_layout(
    legend=dict(orientation='h', yanchor='top', y=1.1, x=0), hovermode='x unified'
)
fig.show()

fig = px.bar(
    df_temp,
    x='day',
    y=df_temp.iloc[:, 1:-1].columns,
    text_auto=True,
    title='Ad cost share per source per day',
    labels=dict(value='Cost, %', day='Date', variable='source'),
    width=FIG_WIDTH * 100,
    height=FIG_HEIGHT * 100,
    template='plotly_white',
)
fig.update_layout(
    legend=dict(orientation='h', yanchor='top', y=1.1, x=0), hovermode='x unified'
)
fig.show()

fig = px.pie(
    round(df_costs, 2),
    values='cost',
    names='source',
    title='Percentage of spend by traffic source',
    width=FIG_WIDTH * 100,
    height=FIG_HEIGHT * 100,
    template='plotly_white',
)
fig.show()


Наши траты на рекламу уменьшались день ко дню, но оставались равномерно распределены по каналам.

Последним упражнением посмотрим, на стоимость привлечения пользователя. Самая большая доля затрат приходится на `yandex_direct` и с этого канала пришло основное количество пользователей (удивительно), но в то же время, хотя затраты на `instagram_new_adverts` сопоставимы с Яндексом, мы явно получаем меньше игроков с этого канала. 

In [18]:
df_temp = (
    pd.merge(
        left=pd.pivot_table(
            data=df_costs,
            index='source',
            values='cost',
            aggfunc='sum',
            margins=True,
        ),
        right=pd.pivot_table(
            data=df_sources,
            index='source',
            values='user_id',
            aggfunc='count',
            margins=True,
        ),
        on='source',
        how='left'
    )
    .reset_index()
    .rename_axis('index', axis=1)
    .set_axis(['source', 'cost_total', 'user_id_count'], axis=1)
    .assign(
        cost_per_user = lambda df: df.cost_total / df.user_id_count,
        sort_key = lambda df: df.source == 'All',
    )
    .sort_values(['sort_key', 'cost_per_user'], ascending=[True, False])
    .drop('sort_key', axis=1)
)

fig = px.bar(
    round(df_temp, 2),
    x='cost_per_user',
    y='source',
    text_auto=True,
    title='Average per user ad cost per source',
    labels=dict(cost_per_user='Cost per user, $/user', source='Traffic source'),
    width=FIG_WIDTH * 100,
    height=FIG_HEIGHT * 100,
    template='plotly_white',
)
fig.update_xaxes(showticklabels=False, showgrid=False)
fig.show()

display(round(df_temp, 2))


,source,cost_total,user_id_count,cost_per_user
0,facebook_ads,2140.90,2726,0.79
1,instagram_new_adverts,2161.44,3347,0.65
2,yandex_direct,2233.11,4817,0.46
3,youtube_channel_reklama,1068.12,2686,0.40
4,All,7603.58,13576,0.56


Как и ожидалось в предыдущей части, у нас есть более эффективные и менее эффективные каналы. Самым дешевым для привлечения пользователей стал YouTube, Yandex находится близко рядом с ним.

Последним упражнением посмотрим, как распределено время в игре по каналам.

In [19]:
df_temp = (
    pd.pivot_table(
        data=df_actions,
        index='user_id',
        values='event_datetime',
        aggfunc=['min', 'max'],
    )
    .reset_index()
    .droplevel(level=1, axis=1)
    .rename_axis('index', axis=1)
    .merge(df_sources, on='user_id', how='left')
    .set_axis(['user_id', 'start_datetime', 'end_datetime', 'source'], axis=1)
    .assign(
        hours_active = lambda df: (df.end_datetime - df.start_datetime).dt.total_seconds() / 60 / 60
    )
)
    
fig = px.violin(
    df_temp,
    y='source',
    x='hours_active',
    # spanmode='hard',
    title='Distribution of # of hours spent by traffic source',
    labels=dict(
        hours_active='Hours active',
        source='Traffic source',
    ),
    width=FIG_WIDTH * 100,
    height=FIG_HEIGHT * 100 * 1.5,
    template='plotly_white',
)
fig.show()
    
display(
    round(
        df_temp.pivot_table(
            index='user_id', columns='source', values='hours_active', aggfunc='sum'
        ).describe().T, 1
    )
)

,count,mean,std,min,25%,50%,75%,max
source,,,,,,,,
facebook_ads,2726.0,257.6,104.2,0.0,182.2,252.9,323.0,727.8
instagram_new_adverts,3347.0,258.9,103.6,0.0,188.9,255.1,321.6,729.6
yandex_direct,4817.0,254.8,104.5,0.0,183.1,251.2,317.2,742.0
youtube_channel_reklama,2686.0,254.1,105.4,0.0,183.2,252.1,318.0,698.0


Мы видим похожие распределения. Видимо, пользователи проводят одинаковое количество времени на первом уровне вне зависимости от рекламного канала.

Учитывая, что затраты на привлечение были распределены равномерно, то в следующей итерации имеет смысл попробовать сфокусировать наши ресурсы на более дешевых каналах, т.к. потенциальная выручка от просмотра рекламы в игре не будет зависеть от того, откуда приходят пользователи: они проводят одинаковое время на первом уровне.

## Провека гипотез

Перейдем к проверке двух гипотез, которые мы определили:

   1. Время завершения уровня различается в зависимости от способа прохождения.
   2. Количество построек различается от канала привлечения.

Выберем уровень статистической значимости `alpha = 0.05`.

### Время завершения уровня

Запишем первую гипотезу как:

> **Нулевая гипотеза (H0):** Нет статистически значимых различий между временами завершения уровня для различных прохождений.

> **Альтернативная гипотеза (H1):** Есть статистически значимые различия между временами завершения уровня для различных прохождений.

Мы принимаем нулевую гипотезу, если `p_value > alpha`. Мы принимаем альтернативную гипотезу, если `p_value < alpha`.

Для начала, соберем датасеты, которые будем использовать: один с пользователями, которые выиграли через PvP, а другой - с пользователями, которые выиграли через PvE.

In [20]:
alpha = 0.05

df_pvp_win = (
    pd.pivot_table(
        data=df_actions,
        index='user_id',
        values='event_datetime',
        aggfunc=['min', 'max'],
    )
    .reset_index()
    .droplevel(level=1, axis=1)
    .rename_axis('index', axis=1)
    .set_axis(['user_id', 'start_datetime', 'end_datetime'], axis=1)
    .assign(
        hours_active = lambda df: (df.end_datetime - df.start_datetime).dt.total_seconds() / 60 / 60
    )
)
# PvE wins are when player completes a project
df_pve_win = (
    df_pvp_win[
        df_pvp_win.user_id.isin(df_actions[df_actions.event == 'project_completed'].user_id)
    ]
).assign(legend = 'pve_win')
# PvP wins are when player completes a stage, but NOT by PvE
df_pvp_win = (
    df_pvp_win[
        ~df_pvp_win.user_id.isin(df_pve_win.user_id)
        & df_pvp_win.user_id.isin(df_actions[df_actions.event == 'stage_1_completed'].user_id)
    ]
).assign(legend = 'pvp_win')


Теперь проверим первую гипотезу.

In [21]:
print(
    'Mean PvE completion time: ', round(df_pve_win.hours_active.mean(), 2), '\n',
    'Mean PvP completion time: ', round(df_pvp_win.hours_active.mean(), 2),
    sep=''
)
fig = px.violin(
    pd.DataFrame(pd.concat([df_pve_win, df_pvp_win])),
    x='hours_active',
    y='legend',
    title='Distribution of # of hours active by win type',
    labels=dict(hours_active='Hours active', legend='Win type'),
    width=FIG_WIDTH * 100,
    height=FIG_HEIGHT * 100 * 1.5,
    template='plotly_white',
)
fig.show()


Mean PvE completion time: 323.01
Mean PvP completion time: 266.51


Похоже, средние значения разнятся. Проверим это статистически.

In [22]:
results = st.ttest_ind(df_pve_win.hours_active, df_pvp_win.hours_active)
print('p-value:', round(results.pvalue, 3))

if results.pvalue < alpha:
    print('Reject null hupothesis')
else:
    print('Accept null hypothesis')


p-value: 0.0
Reject null hupothesis


Действительно разняться. Отвергаем нулевую гипотезу, принимаем альтернативную гипотезу.

### Количеством постороек в игре

Запишем вторую гипотезу как:

> **Нулевая гипотеза (H0):** Нет статистически значимых различий между количеством постороек в игре для различных источников трафика.

> **Альтернативная гипотеза (H1):** Есть статистически значимые различия количеством постороек в игре для различных источников трафика.

Мы принимаем нулевую гипотезу, если `p_value > alpha`. Мы принимаем альтернативную гипотезу, если `p_value < alpha`.

Снова соберем датасет для анализа.

In [23]:
df_buildings_completed = (
    pd.pivot_table(
        data=df_actions[df_actions.event == 'building_completed'],
        index='user_id',
        values='event_datetime',
        aggfunc='count',
    )
    .reset_index()
    .rename_axis('index', axis=1)
    .merge(df_sources, on='user_id', how='left')
    .set_axis(['user_id', 'buildings_count', 'source'], axis=1)
)

fig = px.violin(
    df_buildings_completed,
    x='buildings_count',
    y='source',
    # spanmode='hard',
    title='Distribution of # of buildings by traffic source',
    labels=dict(buildings_count='Buildings, #', source='Traffic source'),
    width=FIG_WIDTH * 100,
    height=FIG_HEIGHT * 100 * 1.5,
    template='plotly_white',
)
fig.show()


Визуально, похоже нет разницы между тем, сколько зданий игроки построили в зависимости от источника трафика. Проверим это формально для Яндекса и Fb.


In [24]:
results = st.ttest_ind(
    df_buildings_completed[df_buildings_completed.source == 'yandex_direct'].buildings_count,
    df_buildings_completed[df_buildings_completed.source == 'facebook_ads'].buildings_count
)
print('p-value:', round(results.pvalue, 3))

if results.pvalue < alpha:
    print('Reject null hupothesis')
else:
    print('Accept null hypothesis')


p-value: 0.01
Reject null hupothesis


Действительно разницы нет. Принимаем нулевую гипотезу.

## Выводы

У нас получилось интересное упражнение:

1. Мы рассмотрели выборку с данными о пользователях с 4 мая по 5 июня (для затрат на рекламу - с 3 по 9 мая). За этот период у нас было 13.5К уникальных пользоваталей и более 135К событий. Эти пользователи пришли к нам из 4 каналов, где мы потратили 7.6К условных единиц денег.


2. Из 13.5К уникальных пользователей 5.8К завершили первый уровень. Большинство (около 2/3) этих игроков прошли уровень через PvP. Остальные игроки завершили `satelite_orbital_assembly`. Это может говорить о некотором дисбалансе в том, как игроки проходят уровень - следует уведомить наших геймдизайнеров, чтобы PvE путь совсем не исчез (а также, чтобы мы заработали больше денег).


3. Хотя затраты по каналам были распеределены почти равномерно, большинство пользователей (около 30%) пришли из Яндекса. Этот канал и YouTube оказались самыми дешвевыми с точки зрения затрт на пользователя (0.4 для YouTube и 0.46 для Яндекса).


4. Мы провели две гипотезы: количество времени в игре от типа победы и колчество построек от источника трафика. В первом случае, игроки провели больше времени в игре, если выиграли через PvE. Во втором случае, количество построек от источника трафика не зависит.


5. В следующей итерации, мы можем пересмотреть наш маркетинг: стоит приоритизировать дешевые каналы в первую очередь (YouTube, Yandex, Instagram, Facebook), т.к. время игроков в приложении не зависит от источника трафика. Это значит и наша потенциальная выручка от рекламы тоже не будет меняться.